<br>

<div align=center><font color=maroon size=6 style="line-height:40px;"><b>Unicode strings</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TensorFlow > <a href="https://www.tensorflow.org/resources" style="text-decoration:none;">Resources</a> 
    * `TensorFlow > Resources > Text > Guide > `<a href="https://www.tensorflow.org/text/guide/unicode" style="text-decoration:none;">Unicode strings</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/guide/unicode.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

## Introduction

NLP models often handle different languages with different character sets.  <font style="color:maroon;font-size:120%">***Unicode*** is a standard encoding system that is used to represent characters from almost all languages.  Every Unicode character is encoded using a unique integer [code point](https://en.wikipedia.org/wiki/Code_point) between `0` and `0x10FFFF`. A *Unicode string* is a sequence of zero or more code points.</font>

This tutorial shows how to represent Unicode strings in TensorFlow and manipulate them using Unicode equivalents of standard string ops. It separates Unicode strings into tokens based on script detection.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
print(tf.__version__)

2.8.0


<br>
<br>
<br>

## The `tf.string` data type
<br>

The basic TensorFlow `tf.string` `dtype` allows you to build tensors of byte strings.<br>
<br>

<font style="color:maroon;font-size:130%">**Unicode strings are [utf-8](https://en.wikipedia.org/wiki/UTF-8) encoded by default.**</font>

<br>

In [5]:
tf.constant("Thanks 😊")

<tf.Tensor: shape=(), dtype=string, numpy=b'Thanks \xf0\x9f\x98\x8a'>

In [6]:
tf.constant(u"Thanks 😊")

<tf.Tensor: shape=(), dtype=string, numpy=b'Thanks \xf0\x9f\x98\x8a'>

<br>

<font style="color:maroon;font-size:130%;line-height:30px">A `tf.string` tensor treats **byte strings** as atomic units. This enables it to store byte strings of varying lengths. The string length is not included in the tensor dimensions.</font>

In [7]:
tf.constant([u"You're", u"welcome!"]).shape

TensorShape([2])

<br>

<font style="color:maroon;font-size:130%;line-height:30px">If you use Python to construct strings, note that [string literals](https://docs.python.org/3/reference/lexical_analysis.html) are Unicode-encoded by default.</font>

<br>
<br>
<br>

## Representing Unicode

<font style="color:maroon;font-size:120%;line-height:30px">There are two standard ways to represent a Unicode string in TensorFlow:

* `string` **scalar** — where the sequence of code points is encoded using a known [character encoding](https://en.wikipedia.org/wiki/Character_encoding).
* `int32` **vector** — where each position contains a single code point.</font>

For example, the following three values all represent the Unicode string `"语言处理"` (which means "language processing" in Chinese):

In [8]:
# Unicode string, represented as a UTF-8 encoded string scalar.
text_utf8 = tf.constant(u"语言处理")
text_utf8

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe8\xaf\xad\xe8\xa8\x80\xe5\xa4\x84\xe7\x90\x86'>

<br>

In [9]:
# Unicode string, represented as a UTF-16-BE encoded string scalar.
text_utf16be = tf.constant(u"语言处理".encode("UTF-16-BE"))
text_utf16be

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8b\xed\x8a\x00Y\x04t\x06'>

<br>

In [10]:
# Unicode string, represented as a vector of Unicode code points.
text_chars = tf.constant([ord(char) for char in u"语言处理"])
text_chars

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35821, 35328, 22788, 29702])>

<br>
<br>

### Converting between representations

<font style="color:maroon;font-size:120%;line-height:30px">TensorFlow provides operations to convert between these different representations:

* `tf.strings.unicode_decode`: Converts an **encoded string** scalar to a vector of **code points**.
* `tf.strings.unicode_encode`: Converts a vector of code points to an encoded string scalar.
* `tf.strings.unicode_transcode`: Converts an encoded string scalar to a different encoding.</font>

In [13]:
print(text_utf8)

tf.strings.unicode_decode(text_utf8,
                          input_encoding='UTF-8')

tf.Tensor(b'\xe8\xaf\xad\xe8\xa8\x80\xe5\xa4\x84\xe7\x90\x86', shape=(), dtype=string)


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35821, 35328, 22788, 29702])>

<br>

In [14]:
print(text_chars)

tf.strings.unicode_encode(text_chars,
                          output_encoding='UTF-8')

tf.Tensor([35821 35328 22788 29702], shape=(4,), dtype=int32)


<tf.Tensor: shape=(), dtype=string, numpy=b'\xe8\xaf\xad\xe8\xa8\x80\xe5\xa4\x84\xe7\x90\x86'>

<br>

In [15]:
tf.strings.unicode_transcode(text_utf8,
                             input_encoding='UTF8',
                             output_encoding='UTF-16-BE')

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8b\xed\x8a\x00Y\x04t\x06'>

<br>
<br>

### Batch dimensions

<font style="color:maroon;font-size:120%;line-height:30px">When decoding multiple strings, the number of characters in each string may not be equal.  The return result is a [`tf.RaggedTensor`](https://colab.research.google.com/github/tensorflow/text/blob/master/guide/ragged_tensor.ipynb), where the **innermost** dimension length varies depending on the number of characters in each string.</font>

In [16]:
# A batch of Unicode strings, each represented as a UTF8-encoded string.
batch_utf8 = [s.encode('UTF-8') for s in 
              [u'hÃllo', u'What is the weather tomorrow', u'Göödnight', u'😊']]
print(batch_utf8)
print()

batch_chars_ragged = tf.strings.unicode_decode(batch_utf8,
                                               input_encoding='UTF-8')
print(batch_chars_ragged)
print()

for sentence_chars in batch_chars_ragged.to_list():
    print(sentence_chars)

[b'h\xc3\x83llo', b'What is the weather tomorrow', b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a']

<tf.RaggedTensor [[104, 195, 108, 108, 111],
 [87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116,
  104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]               ,
 [71, 246, 246, 100, 110, 105, 103, 104, 116], [128522]]>

[104, 195, 108, 108, 111]
[87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116, 104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]
[71, 246, 246, 100, 110, 105, 103, 104, 116]
[128522]


<br>

<font style="color:maroon;font-size:120%;line-height:30px">You can use this `tf.RaggedTensor` directly, or convert it to a dense `tf.Tensor` with padding or a `tf.sparse.SparseTensor` using the methods `tf.RaggedTensor.to_tensor` and `tf.RaggedTensor.to_sparse`.</font>

In [17]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

[[   104    195    108    108    111     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [    87    104     97    116     32    105    115     32    116    104
     101     32    119    101     97    116    104    101    114     32
     116    111    109    111    114    114    111    119]
 [    71    246    246    100    110    105    103    104    116     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [128522     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]]


<br>

In [20]:
batch_chars_sparse = batch_chars_ragged.to_sparse()
print(batch_chars_sparse)
print()

print(batch_chars_sparse.dense_shape)
print()

nrows, ncols = batch_chars_sparse.dense_shape.numpy()  # [ 4 28]
elements = [['_' for i in range(ncols)] for j in range(nrows)]

for (row, col), value in zip(batch_chars_sparse.indices.numpy(), 
                             batch_chars_sparse.values.numpy()):
    elements[row][col] = str(value)

# max_width = max(len(value) for row in elements for value in row)
value_lengths = []
for row in elements:
    for value in row:
        value_lengths.append(len(value))

max_width = max(value_lengths)
print('[%s]' % '\n '.join('[%s]' % ', '.join(value.rjust(max_width) for value in row)
                          for row in elements))

SparseTensor(indices=tf.Tensor(
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 0  3]
 [ 0  4]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 1  3]
 [ 1  4]
 [ 1  5]
 [ 1  6]
 [ 1  7]
 [ 1  8]
 [ 1  9]
 [ 1 10]
 [ 1 11]
 [ 1 12]
 [ 1 13]
 [ 1 14]
 [ 1 15]
 [ 1 16]
 [ 1 17]
 [ 1 18]
 [ 1 19]
 [ 1 20]
 [ 1 21]
 [ 1 22]
 [ 1 23]
 [ 1 24]
 [ 1 25]
 [ 1 26]
 [ 1 27]
 [ 2  0]
 [ 2  1]
 [ 2  2]
 [ 2  3]
 [ 2  4]
 [ 2  5]
 [ 2  6]
 [ 2  7]
 [ 2  8]
 [ 3  0]], shape=(43, 2), dtype=int64), values=tf.Tensor(
[   104    195    108    108    111     87    104     97    116     32
    105    115     32    116    104    101     32    119    101     97
    116    104    101    114     32    116    111    109    111    114
    114    111    119     71    246    246    100    110    105    103
    104    116 128522], shape=(43,), dtype=int32), dense_shape=tf.Tensor([ 4 28], shape=(2,), dtype=int64))

tf.Tensor([ 4 28], shape=(2,), dtype=int64)

[[   104,    195,    108,    108,    111,      _,      _,      _,      _,      _,      _,     

<br>

<font style="color:maroon;font-size:130%;line-height:30px">When encoding multiple strings with the **same lengths**, use a **`tf.Tensor`** as the input.</font>

In [21]:
tf.strings.unicode_encode([[99, 97, 116], [100, 111, 103], [99, 111, 119]],
                          output_encoding='UTF-8')

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'cat', b'dog', b'cow'], dtype=object)>

<br>

<font style="color:maroon;font-size:130%;line-height:30px">When encoding multiple strings with **varying length**, use a **`tf.RaggedTensor`** as the input.</font>

In [24]:
batch_chars_ragged

<tf.RaggedTensor [[104, 195, 108, 108, 111],
 [87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116,
  104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]               ,
 [71, 246, 246, 100, 110, 105, 103, 104, 116], [128522]]>

In [22]:
tf.strings.unicode_encode(batch_chars_ragged, output_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

<br>

<font style="color:maroon;font-size:130%;line-height:30px">If you have a tensor with multiple strings **in padded or sparse format**, convert it first into a `tf.RaggedTensor` before calling `tf.strings.unicode_encode`.</font>

In [23]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

<br>

In [25]:
tf.strings.unicode_encode(tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1),
                          output_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

<br>
<br>
<br>

## Unicode operations

### Character length

<font style="color:maroon;font-size:120%;line-height:30px">Use the `unit` parameter of the `tf.strings.length` op to indicate how **character lengths** should be computed.<br>

**`unit` defaults to `"BYTE"`**, but it can be set to other values, such as `"UTF8_CHAR"` or `"UTF16_CHAR"`, to determine the number of Unicode codepoints in each encoded string.</font>

In [27]:
# Note that the final character takes up 4 bytes in UTF8.
thanks = u'Thanks 😊'.encode('UTF-8')
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} bytes; \n{} UTF-8 characters'.format(num_bytes, num_chars))

# 对于 UTF8_CHAR 来说：
# "Thanks" 有 6 个 byte；6 个 character。
# " " 有 1 个 byte；1 个 character。
# 😊 有 4 个 byte；1 个 character。

11 bytes; 
8 UTF-8 characters


In [30]:
thanks

b'Thanks \xf0\x9f\x98\x8a'

<br>
<br>

### Character substrings

<font style="color:maroon;font-size:120%;line-height:30px">The `tf.strings.substr` op accepts the `unit` parameter, and uses it to determine what kind of **offsets** the **`pos`** and `len` paremeters contain.</font>

In [28]:
# Here, unit='BYTE' (default). Returns a single byte with len=1
tf.strings.substr(thanks, pos=7, len=1).numpy()

# 此时第 7 个 byte 之后的第一个 byte 恰好是 😊 的 第一个 byte。
# 

b'\xf0'

In [29]:
# Specifying unit='UTF8_CHAR', returns a single 4 byte character in this case
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

# 此时第 7 个 character 之后的第一个 character 恰好是 😊。

b'\xf0\x9f\x98\x8a'


In [31]:
# Specifying unit='UTF8_CHAR', returns a single 4 byte character in this case
print(tf.strings.substr(thanks, pos=5, len=1, unit='UTF8_CHAR').numpy())

# 此时第 5 个 character 之后的第一个 character 恰好是 "Thanks 😊" 中的 s。

b's'


<br>
<br>
<br>

### Split Unicode strings

<font style="color:maroon;font-size:120%;line-height:30px">The `tf.strings.unicode_split` operation splits unicode strings into substrings of **individual characters**.</font>

In [32]:
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

array([b'T', b'h', b'a', b'n', b'k', b's', b' ', b'\xf0\x9f\x98\x8a'],
      dtype=object)

<br>
<br>

### Byte offsets for characters

<font style="color:maroon;font-size:120%;line-height:30px">To align the character tensor generated by `tf.strings.unicode_decode` with the original string, it's useful to know the offset for where each character begins.  The method `tf.strings.unicode_decode_with_offsets` is similar to `unicode_decode`, except that it returns a second tensor containing the start offset of each character.</font>

In [33]:
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u'🎈🎉🎊', 'UTF-8')

for (codepoint, offset) in zip(codepoints.numpy(), offsets.numpy()):
    print('At byte offset {}: codepoint {}'.format(offset, codepoint))

At byte offset 0: codepoint 127880
At byte offset 4: codepoint 127881
At byte offset 8: codepoint 127882


<br>
<br>
<br>

## Unicode scripts

<font style="color:maroon;font-size:120%;line-height:30px">Each Unicode code point belongs to a single collection of codepoints known as a [**script**](https://en.wikipedia.org/wiki/Script_%28Unicode%29) .  **A *character's script* is helpful in determining which language the character might be in**.<br>
For example, knowing that 'Б' is in Cyrillic script indicates that modern text containing that character is likely from a Slavic language such as Russian or Ukrainian.

TensorFlow provides the `tf.strings.unicode_script` operation to determine which script a given codepoint uses. The script codes are `int32` values corresponding to [International Components for
Unicode](http://site.icu-project.org/home) (ICU) [`UScriptCode`](http://icu-project.org/apiref/icu4c/uscript_8h.html) values.</font>

In [34]:
uscript = tf.strings.unicode_script([33464, 1041])  # ['芸', 'Б']

print(uscript.numpy())  # [17, 8] == [USCRIPT_HAN, USCRIPT_CYRILLIC]

[17  8]


<br>

The `tf.strings.unicode_script` operation can also be applied to multidimensional `tf.Tensor`s or `tf.RaggedTensor`s of codepoints:

In [35]:
print(tf.strings.unicode_script(batch_chars_ragged))

<tf.RaggedTensor [[25, 25, 25, 25, 25],
 [25, 25, 25, 25, 0, 25, 25, 0, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25,
  0, 25, 25, 25, 25, 25, 25, 25, 25]                                      ,
 [25, 25, 25, 25, 25, 25, 25, 25, 25], [0]]>


In [38]:
print(batch_chars_ragged)
print()
print(tf.strings.unicode_encode(batch_chars_ragged, output_encoding='UTF-8'))

<tf.RaggedTensor [[104, 195, 108, 108, 111],
 [87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116,
  104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]               ,
 [71, 246, 246, 100, 110, 105, 103, 104, 116], [128522]]>

tf.Tensor(
[b'h\xc3\x83llo' b'What is the weather tomorrow'
 b'G\xc3\xb6\xc3\xb6dnight' b'\xf0\x9f\x98\x8a'], shape=(4,), dtype=string)


<br>
<br>
<br>

## Example: Simple segmentation

<font style="color:maroon;font-size:120%;line-height:30px">Segmentation is the task of splitting text into word-like units. This is often easy when space characters are used to separate words, but some languages (like Chinese and Japanese) do not use spaces, and some languages (like German) contain long compounds that must be split in order to analyze their meaning. In web text, different languages and scripts are frequently mixed together, as in "NY株価" (New York Stock Exchange).

We can perform very rough segmentation (without implementing any ML models) by using changes in script to approximate word boundaries. This will work for strings like the "NY株価" example above. It will also work for most languages that use spaces, **as the *space characters* of various scripts are all classified as USCRIPT_COMMON, a special script code that differs from that of any actual text**.</font>

In [39]:
# dtype: string; shape: [num_sentences]
#
# The sentences to process.  Edit this line to try out different inputs!
sentence_texts = [u'Hello, world.', u'世界こんにちは']

<br>

First, decode the sentences into character codepoints, and find the script identifeir for each character.

In [40]:
# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_codepoint[i, j] 
# is the codepoint for the j'th character in the i'th sentence.
sentence_char_codepoint = tf.strings.unicode_decode(sentence_texts, 'UTF-8')
print(sentence_char_codepoint)
print()


sentence_char_script = tf.strings.unicode_script(sentence_char_codepoint)
print(sentence_char_script)

<tf.RaggedTensor [[72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 46],
 [19990, 30028, 12371, 12435, 12395, 12385, 12399]]>

<tf.RaggedTensor [[25, 25, 25, 25, 25, 0, 0, 25, 25, 25, 25, 25, 0],
 [17, 17, 20, 20, 20, 20, 20]]>


<br>

<font style="color:maroon;font-size:120%;line-height:30px">Use the **script identifiers** to determine where word boundaries should be added.</font>  Add a word boundary at the beginning of each sentence, and for each character whose script differs from the previous character.

In [41]:
# dtype: bool; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_starts_word[i, j] is True if the j'th character in the i'th
# sentence is the start of a word.
sentence_char_starts_word = tf.concat([ tf.fill([sentence_char_script.nrows(), 1], True),
                                        tf.not_equal(sentence_char_script[:, 1:], 
                                                     sentence_char_script[:, :-1])],
                                        axis=1)

# dtype: int64; shape: [num_words]
#
# word_starts[i] is the index of the character that starts the i'th word (in
# the flattened list of characters from all sentences).
word_starts = tf.squeeze(tf.where(sentence_char_starts_word.values), axis=1)
print(word_starts)

tf.Tensor([ 0  5  7 12 13 15], shape=(6,), dtype=int64)


<br>

You can then use those start offsets to build a `RaggedTensor` containing the list of words from all batches.

In [42]:
# dtype: int32; shape: [num_words, (num_chars_per_word)]
#
# word_char_codepoint[i, j] is the codepoint for the j'th character in the i'th word.
word_char_codepoint = tf.RaggedTensor.from_row_starts(values=sentence_char_codepoint.values,
                                                      row_starts=word_starts)
print(word_char_codepoint)

<tf.RaggedTensor [[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46],
 [19990, 30028], [12371, 12435, 12395, 12385, 12399]]>


<br>

To finish, segment the word codepoints `RaggedTensor` back into sentences and encode into UTF-8 strings for readability.

In [43]:
# dtype: int64; shape: [num_sentences]
#
# sentence_num_words[i] is the number of words in the i'th sentence.
sentence_num_words = tf.reduce_sum(
    tf.cast(sentence_char_starts_word, tf.int64),
    axis=1)

# dtype: int32; shape: [num_sentences, (num_words_per_sentence), (num_chars_per_word)]
#
# sentence_word_char_codepoint[i, j, k] is the codepoint for the k'th character
# in the j'th word in the i'th sentence.
sentence_word_char_codepoint = tf.RaggedTensor.from_row_lengths(values=word_char_codepoint,
                                                                row_lengths=sentence_num_words)
print(sentence_word_char_codepoint)

tf.strings.unicode_encode(sentence_word_char_codepoint, 'UTF-8').to_list()

<tf.RaggedTensor [[[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46]],
 [[19990, 30028], [12371, 12435, 12395, 12385, 12399]]]>


[[b'Hello', b', ', b'world', b'.'],
 [b'\xe4\xb8\x96\xe7\x95\x8c',
  b'\xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf']]

<br>
<br>
<br>